In [1]:
import pandas as pd
import numpy as np

#### Import Yearly Data Sets

In [2]:
# Aggregated moving violations data concatted in previous notebook
mv2011 = pd.read_csv('Data/Moving Violation data year/moving_violations2011.csv')
mv2012 = pd.read_csv('Data/Moving Violation data year/moving_violations2012.csv')
mv2013 = pd.read_csv('Data/Moving Violation data year/moving_violations2013.csv')
mv2014 = pd.read_csv('Data/Moving Violation data year/moving_violations2014.csv')
mv2015 = pd.read_csv('Data/Moving Violation data year/moving_violations2015.csv')

In [289]:
# Data example head
mv2012.head(2)

,Unnamed: 0,LONGITUDE,LATITUDE,TICKETTYPE,TICKETISSUEDATE,TRAP
0,0,-77.088404,38.915716,Photo,2012-01-02T00:00:00.000Z,1
1,1,-76.952187,38.895681,Photo,2012-01-02T00:00:00.000Z,1


#### Import Crash Data set

In [3]:
crashes = pd.read_csv('Data/Crashes_data.csv')

#### Merge all Yearly ticket data into one data frame

In [4]:
tickets = mv2011.append([mv2012, mv2013, mv2014, mv2015])

In [5]:
# Drop extra row
tickets.drop('Unnamed: 0', axis =1, inplace = True)

In [6]:
# Only take rows where the ticket type is 'Photo' (engineer column 'TRAP' also does the same)
traps = tickets[tickets['TRAP'] == 1]

 What is the proportion of Photo Tickets to Hand Written Tickets?

In [7]:
float(len(traps))/float(len(tickets))

0.9356710367210453

93.5% of all tickets given from 2011 to 2015 are from photo traps.

#### Removing the  Time feature from tickets so I am left with only a date
Counts of daily occurances are the lowest level by which this model is going to assess the date.

In [ ]:
ticketdate_list = traps['TICKETISSUEDATE'].tolist()

dtl = []
for date in ticketdate_list:
    new = date.replace('T00:00:00.000Z','')

    dtl.append(new)
    
traps['DATE'] = dtl

traps.drop('TICKETISSUEDATE', axis = 1, inplace = True)

In [ ]:
crash = crashes[['LONGITUDE','LATITUDE','DATE', 'EVENT']]

# Converting the date time object to just a date column. 
crash_t = pd.DataFrame(crash['DATE'].str.split(' ',1).tolist(), columns = ['date','time'])

day = crash_t['date']

crash.drop('DATE', axis = 1, inplace = True)

crash['DATE']= day

# Dropping null values
crash.dropna(inplace = True)

# removing dates outside the range of 2011 - 2015
crash['DATE'] = pd.to_datetime(crash['DATE'])
crash = crash[(crashes['DATE'].dt.year >= 2011)]
crash = crash[(crashes['DATE'].dt.year <= 2015)]

# dont care about the error and depreciation message.  It did what i needed it to.

# -------------------------------------------------------------------------------------

In [ ]:
# adding a 'Crash' feature so i can concat easily
traps.drop('TICKETTYPE',axis = 1, inplace = True)
traps['CRASH'] = 0

In [ ]:
# adding a 'Trap' feature so i can concat easily
crash['EVENT'] = 1
crash.rename(columns = {'EVENT':'CRASH'}, inplace = True)
crash['TRAP'] = 0

Both of my dataframes have the same column structure.  Now I can add them together.

In [14]:
theory = pd.concat([crash, traps])

truncate lat and longitude for the purpose of this experiment and then do a grouped by count

In [16]:
# Date Column needed an adjustment
theory['DATE'] = pd.to_datetime(theory['DATE'], format = '%Y-%m-%d')

In [18]:
# exporting data to attempt a DB scan using AWS considering how long it would take.
theory.to_csv('dbscan_data.csv')

In [ ]:
# round datetime if needed.  
#theory[['LONGITUDE','LATITUDE']] = theory[['LONGITUDE','LATITUDE']].round(3)

In [27]:
# Creating individual sub dataframes by year.  
theory2011 = theory[(theory['DATE'].dt.year == 2011)]
theory2012 = theory[(theory['DATE'].dt.year == 2012)]
theory2013 = theory[(theory['DATE'].dt.year == 2013)]
theory2014 = theory[(theory['DATE'].dt.year == 2014)]
theory2015 = theory[(theory['DATE'].dt.year == 2015)]

In [72]:
# Checking prior
theory['CRASH'].value_counts()

0    2958211
1     123070
Name: CRASH, dtype: int64

In [ ]:
event_count = theory.groupby(['LONGITUDE', 'LATITUDE','DATE'],as_index = False).count()

 Looks like this was a failure.  No way 1740 crashes occured on one day.  Its just counting all the observations everyday.  

# Granger Causaility Test
My Kernal Died a few times while troubleshooting this function so i included all the importand and necessary cleaning needed in the two cells below.
I also ran though a trial experiment with one year of data.  

In [120]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import grangercausalitytests

In [121]:
theory = pd.read_csv('dbscan_data.csv')
theory.drop('Unnamed: 0', axis =1, inplace = True)
theory['DATE'] = pd.to_datetime(theory['DATE'], format = '%Y-%m-%d')

In [ ]:
# 2011 Trial.  This runs pretty fast im going to up it to 5 years of data.  
# theory2011 = theory[(theory['DATE'].dt.year == 2011)]

# theory2011['LATITUDE'] = theory2011['LATITUDE'].round(3)
# theory2011['LATITUDE'] = theory2011['LATITUDE'].astype(str)

# theory2011['LONGITUDE'] = theory2011['LONGITUDE'].round(3)
# theory2011['LONGITUDE'] = theory2011['LONGITUDE'].astype(str)

# theory2011['location'] = theory2011['LATITUDE']+', '+theory2011['LONGITUDE']

In [171]:
# Per Joe and Haley I need to aggregate by the month
theory['DATE'] = theory.DATE.map(lambda x: x.strftime('%Y-%m'))
theory.head()

,CRASH,DATE,LATITUDE,LONGITUDE,TRAP,location
0,1,2009-01,38.888,-76.932,0,"38.888, -76.932"
1,1,2009-01,38.933,-76.973,0,"38.933, -76.973"
2,1,2008-08,38.906,-77.045,0,"38.906, -77.045"
3,1,2008-08,38.887,-76.937,0,"38.887, -76.937"
4,1,2009-06,38.927,-77.034,0,"38.927, -77.034"


In [130]:
theory['LATITUDE'] = theory['LATITUDE'].round(3)
theory['LATITUDE'] = theory['LATITUDE'].astype(str)

theory['LONGITUDE'] = theory['LONGITUDE'].round(3)
theory['LONGITUDE'] = theory['LONGITUDE'].astype(str)

theory['location'] = theory['LATITUDE']+', '+theory['LONGITUDE']

In [174]:
locations_ua = theory['location'].unique()

In [152]:
# locations_u = theory2011['location'].unique()

In [271]:
COLS = ['Location',
        'lag1_Ftest','lag1_CHI2','lag1_LR','lag1_params_Ftest',
        'lag2_Ftest','lag2_CHI2','lag2_LR','lag2_params_Ftest',
        'lag3_Ftest','lag3_CHI2','lag3_LR','lag3_params_Ftest',
        'lag4_Ftest','lag4_CHI2','lag4_LR','lag4_params_Ftest',
        'lag5_Ftest','lag5_CHI2','lag5_LR','lag5_params_Ftest',
        'lag6_Ftest','lag6_CHI2','lag6_LR','lag6_params_Ftest']
granger_results_df = pd.DataFrame(columns = COLS)

In [272]:
# already is a list of locations that have ALREADY been passed through the function and if they
# were good values it will have already populated the dataframe and bad ones will be there to
#skip over
already=[]

#### GCT Function
- This function pulls from my dataframe of all ticket and crash records based on a list of unique locations (String of Lat and Lon).
- Checks to make sure there is at least one crash record and at least 2 ticket records.
- Splits data into two dataframes. One for Crashes and one for Traps
- Drops any remaining zero value records.
- Counts up the observations to aggregate at the monthly level.
- Sorts DFs by data.
- Rejoins or date and sorts again on date
- Passes resulting dataframe through the StatsModel Granger Causality Test.
- Appends results to a Dataframe

Sometimes values make it through that can not be passed through the GCT and cause the loop to distrupt.  
This occurs if there are not enough observations to calculate the max lag (only 5 months of observations trying to calculate a cause on 6 months of lag.  I created a work around that appends every location that has been passed to a list and then run a 'if x is not in y_list:' proceed with function type deal so i could just keep running the GCT over and over and over again and appending new results to my dataframe.  

In [273]:
def gct_cal(trial):
    # use location to pull all relevant observations from teh Theory2011 DF
    trial_loc_df = theory[['DATE','CRASH','TRAP']][(theory['location'] == trial)]
    already.append(trial)
    # check if the location has values for both 'CRASH' and 'TRAP'
    if trial_loc_df['CRASH'].sum() > 0:

        if trial_loc_df['TRAP'].sum() > 1:
            print(trial)
            # Split dataframe
            trial_df1 = pd.DataFrame(trial_loc_df[['DATE','CRASH']])
            trial_df2 = pd.DataFrame(trial_loc_df[['DATE','TRAP']])
            
            # dropping all values with no record
            trial_df1a =trial_df1[trial_df1['CRASH'] != 0]
            trial_df2a =trial_df2[trial_df2['TRAP'] != 0]
            
            # going to have to get daily counts to make indexes unique values by taking the counts for each day
            crash_loc = trial_df1a.groupby('DATE').count()
            trap_loc = trial_df2a.groupby('DATE').count()
            
            # remerging dataframes together and filling nulls with 0's
            # Switch around the position of 'crash_loc' and 'trap_loc' to see the result of one on the other.
            trial_result = pd.concat([crash_loc,trap_loc], axis =1)
            trial_result.fillna(value = 0, inplace = True)
            
            trial_result.sort_index(inplace = True)
            if len(trial_result) >= 6:
    
            #running the GCT
                gct = grangercausalitytests(trial_result, maxlag = 6)
             
            
            # getting metric values to append to a dataframe
                locale = trial
                lag1_ssr_Ftest = gct[1][0]['ssr_ftest']
                lag1_ssr_CHI2 = gct[1][0]['ssr_chi2test']
                lag1_LR = gct[1][0]['lrtest']
                lag1_params_Ftest = gct[1][0]['params_ftest']

                lag2_ssr_Ftest = gct[2][0]['ssr_ftest']
                lag2_ssr_CHI2 = gct[2][0]['ssr_chi2test']
                lag2_LR = gct[2][0]['lrtest']
                lag2_params_Ftest = gct[2][0]['params_ftest']

                lag3_ssr_Ftest = gct[3][0]['ssr_ftest']
                lag3_ssr_CHI2 = gct[3][0]['ssr_chi2test']
                lag3_LR = gct[3][0]['lrtest']
                lag3_params_Ftest = gct[3][0]['params_ftest']
            
                lag4_ssr_Ftest = gct[4][0]['ssr_ftest']
                lag4_ssr_CHI2 = gct[4][0]['ssr_chi2test']
                lag4_LR = gct[4][0]['lrtest']
                lag4_params_Ftest = gct[4][0]['params_ftest']
            
                lag5_ssr_Ftest = gct[5][0]['ssr_ftest']
                lag5_ssr_CHI2 = gct[5][0]['ssr_chi2test']
                lag5_LR = gct[5][0]['lrtest']
                lag5_params_Ftest = gct[3][0]['params_ftest']
            
                lag6_ssr_Ftest = gct[6][0]['ssr_ftest']
                lag6_ssr_CHI2 = gct[6][0]['ssr_chi2test']
                lag6_LR = gct[6][0]['lrtest']
                lag6_params_Ftest = gct[6][0]['params_ftest']
            
                # appending information to dataframe
                granger_results_df.loc[len(granger_results_df)]=[locale, 
                                                            lag1_ssr_Ftest, lag1_ssr_CHI2, lag1_LR, lag1_params_Ftest,
                                                            lag2_ssr_Ftest, lag2_ssr_CHI2, lag2_LR, lag2_params_Ftest,
                                                            lag3_ssr_Ftest, lag3_ssr_CHI2, lag3_LR, lag3_params_Ftest,
                                                            lag4_ssr_Ftest, lag4_ssr_CHI2, lag4_LR, lag4_params_Ftest,
                                                            lag5_ssr_Ftest, lag5_ssr_CHI2, lag5_LR, lag5_params_Ftest,
                                                            lag6_ssr_Ftest, lag6_ssr_CHI2, lag6_LR, lag6_params_Ftest] 

In [286]:
# For loop to runmy GCT function.
for trial in locations_ua:
    # looks to see if location has already been run.
    if trial not in already:
        gct_cal(trial)

38.892, -77.028

Granger Causality
('number of lags (no zero)', 1)
ssr based F test:         F=0.2299  , p=0.6333  , df_denom=63, df_num=1
ssr based chi2 test:   chi2=0.2408  , p=0.6236  , df=1
likelihood ratio test: chi2=0.2404  , p=0.6239  , df=1
parameter F test:         F=0.2299  , p=0.6333  , df_denom=63, df_num=1

Granger Causality
('number of lags (no zero)', 2)
ssr based F test:         F=0.1044  , p=0.9010  , df_denom=60, df_num=2
ssr based chi2 test:   chi2=0.2262  , p=0.8931  , df=2
likelihood ratio test: chi2=0.2258  , p=0.8932  , df=2
parameter F test:         F=0.1044  , p=0.9010  , df_denom=60, df_num=2

Granger Causality
('number of lags (no zero)', 3)
ssr based F test:         F=1.3013  , p=0.2829  , df_denom=57, df_num=3
ssr based chi2 test:   chi2=4.3834  , p=0.2229  , df=3
likelihood ratio test: chi2=4.2399  , p=0.2367  , df=3
parameter F test:         F=1.3013  , p=0.2829  , df_denom=57, df_num=3

Granger Causality
('number of lags (no zero)', 4)
ssr based F test: 

ValueError: Insufficient observations. Maximum allowable lag is 3

In [288]:
# Exporting to CSV
granger_results_df.to_csv('gct_lag6_cvt.csv')
granger_results_df.to_csv('gct_lag6_tvc.csv')
#gct_lag6_cvt.csv, gct_lag6_tvc.csv

# ------------------------------------------------------------------------------------
# Geopy
This is my attempt to use Geopy's Geolocator to apply Geopy's unique location ID to locations rather than using my rounding meathod to create areas.

Unfortunately it was very slow/ computationally expensive even when run on my AWS instance.  

In [28]:
from geopy.geocoders import Nominatim

In [33]:
geolocator = Nominatim()

local1 = geolocator.reverse('38.888110, -76.931799')
local2 = geolocator.reverse('38.888111, -76.931800')
local3 = geolocator.reverse('38.888000, -76.932000')

In [35]:
print local1.raw['osm_id']
print local2.raw['osm_id']
print local3.raw['osm_id']

55412802
55412802
55412831


In [ ]:
dataset.dict[key]

In [83]:
local1.raw

{u'address': {u'city': u'Washington',
  u'country': u'United States of America',
  u'country_code': u'us',
  u'house': u'Marshall Heights',
  u'house_number': u'4907',
  u'postcode': u'20019',
  u'road': u'A Street Southeast',
  u'state': u'District of Columbia'},
 u'boundingbox': [u'38.8880159', u'38.888188', u'-76.9319524', u'-76.93164'],
 u'display_name': u'4907, A Street Southeast, Marshall Heights, Washington, District of Columbia, 20019, United States of America',
 u'lat': u'38.88808215',
 u'licence': u'Data \xa9 OpenStreetMap contributors, ODbL 1.0. http://www.openstreetmap.org/copyright',
 u'lon': u'-76.931801419496',
 u'osm_id': u'55412802',
 u'osm_type': u'way',
 u'place_id': u'77855796'}

OMS ID stays true down to the fifth decimol

# -------------------------------------------------------------------------------------
# DBSCAN
Similar to Geopy, 6 months of data was possible for DBSCAN  but, 1 year was too much data and would have taken over a day on my AWS instance as well.

Running the 6 month DBSCAN was possible on my local machine however, calculating the Sillhouette score caused some serious issues on my machine killing my kernel.

lets see if we can create dbscan clusters and then reasign those back onto the list of values.

In [82]:
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

for computations sake I am only going to take six months of data for the trial.

In [85]:
theory_2 = theory[(theory['DATE'].dt.year == 2011)]
theory_2 = theory_2[(theory_2['DATE'].dt.month <7)]

In [88]:
X = theory_2[['LONGITUDE','LATITUDE']]

DB = DBSCAN(algorithm = 'auto', eps= 0.001, min_samples = 4 , metric = 'manhattan')

In [90]:
dbs = DB.fit(X)

In [ ]:
core_samples_mask = np.zeros_like(dbs.labels_, dtype=bool)
core_samples_mask[dbs.core_sample_indices_] = True
labels = dbs.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

print('Estimated number of clusters: %d' % n_clusters_)


print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels))

Estimated number of clusters: 474
